In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Mofid/

/content/drive/MyDrive/Mofid


In [ ]:
#!unzip train.zip

In [ ]:
%cd train-1

/content/drive/MyDrive/Mofid/train-1


In [ ]:
!pip install fasttext

In [ ]:
import fasttext
import re
from nltk.corpus import stopwords
import numpy as np, pandas as pd
from gensim.utils import simple_preprocess
import pandas as pd

In [ ]:
!pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
df_train = pd.read_csv('train.csv')
df_train.head()

,Text,Class
0,I do.,0
1,What if anything was decided about whether I s...,0
2,H: If Roubini is right and he's been mostly ri...,0
3,HRC: Below is an oped on the National Security...,0
4,DQoNCg0KDQoNCg0KDQpHb29kIERheSwNCk1heSBpdCBub3...,1


In [ ]:
df_train['Class'] = df_train['Class'].apply(str)
df_train['Text'] = df_train['Text'].apply(str)
df_train.head()

,Text,Class
0,I do.,0
1,What if anything was decided about whether I s...,0
2,H: If Roubini is right and he's been mostly ri...,0
3,HRC: Below is an oped on the National Security...,0
4,DQoNCg0KDQoNCg0KDQpHb29kIERheSwNCk1heSBpdCBub3...,1


In [ ]:
df_train = df_train[['Text', 'Class']].rename(columns = {'Text': 'questions', 'Class': 'category'})
df_train.head()

,questions,category
0,I do.,0
1,What if anything was decided about whether I s...,0
2,H: If Roubini is right and he's been mostly ri...,0
3,HRC: Below is an oped on the National Security...,0
4,DQoNCg0KDQoNCg0KDQpHb29kIERheSwNCk1heSBpdCBub3...,1


In [ ]:
df_train.dtypes

questions    object
category     object
dtype: object

In [ ]:
df_train['questions'].fillna('').astype(str)

0                                                   I do.
1       What if anything was decided about whether I s...
2       H: If Roubini is right and he's been mostly ri...
3       HRC: Below is an oped on the National Security...
4       DQoNCg0KDQoNCg0KDQpHb29kIERheSwNCk1heSBpdCBub3...
                              ...                        
9538                Assume jake and pir talked to u about
9539     Experience in Southeast Asia [John Carland]below
9540    ATTN=3AC=2EE=2EO=2FPRESIDENT=2E I know this me...
9541    Dear Friend,How are you doing? I would be very...
9542    COMPLIMENTS OF THE SEASON=2C  With due respect...
Name: questions, Length: 9543, dtype: object

In [ ]:
# NLP Preprocess
df_train.iloc[:, 0] = df_train.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))
df_train.iloc[:, 0] = df_train.iloc[:, 0].apply(lambda x: ' '.join(w for w in nltk.wordpunct_tokenize(x) if w.lower() in words or not w.isalpha()))
df_train.iloc[:, 0] = df_train.iloc[:, 0].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df_train.head()

,questions,category
0,,0
1,anything decided whether call traffic read say...,0
2,right mostly right far one without additional ...,0
3,national security strategy wrote tomorrow outl...,0
4,mig aw aw aw mig st aw nig aw aw ly aw ha gaw,1


In [ ]:
# Prefixing each row of the category column with '__label__'
df_train.iloc[:, 1] = df_train.iloc[:, 1].apply(lambda x: '__label__' + x)
df_train.head()

,questions,category
0,,__label__0
1,anything decided whether call traffic read say...,__label__0
2,right mostly right far one without additional ...,__label__0
3,national security strategy wrote tomorrow outl...,__label__0
4,mig aw aw aw mig st aw nig aw aw ly aw ha gaw,__label__1


In [ ]:
df_train[['category', 'questions']].to_csv('train.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          escapechar = " ")

***
Test


In [ ]:
df_test = pd.read_csv('test.csv')
df_test.head(2)

,Text
0,"FROM BONGO WALEJOHANNESBURG,SOUTH AFRICA.TELL:..."
1,MR=2E DONALD COLLINSCREDIT MUTUEL DU SENEGAL B...


In [ ]:
df_test = df_test[['Text']].rename(columns = {'Text': 'questions'})
df_test.head(2)

,questions
0,"FROM BONGO WALEJOHANNESBURG,SOUTH AFRICA.TELL:..."
1,MR=2E DONALD COLLINSCREDIT MUTUEL DU SENEGAL B...


In [ ]:
df_test.iloc[:, 0] = df_test.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))
df_test.iloc[:, 0] = df_test.iloc[:, 0].apply(lambda x: ' '.join(w for w in nltk.wordpunct_tokenize(x) if w.lower() in words or not w.isalpha()))
df_test.iloc[:, 0] = df_test.iloc[:, 0].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df_test.head()

,questions
0,bongo south tell dear sir bongo wale son wale ...
1,bank attention collins south working credit ma...
2,meant write tough critic
3,well well
4,dear friend season like send best good health ...


***
Model

In [ ]:
# Training the fastText classifier
model = fasttext.train_supervised('train.txt')

In [ ]:
# Predicting 
preds = []
for i in range(0,len(df_test)):
  pred = model.predict(df_test.iloc[i,0])
  preds.append(pred[1][0])

In [ ]:
out = pd.DataFrame(preds)
out.to_csv('submit.csv', index=False, header=['Class'])